# A network of hashtags

In [ ]:
!pip install minet networkx ipysigma

In [2]:
from minet.twitter import TwitterAPIScraper
import networkx as nx
from ipysigma import Sigma

## Trying ipysigma

In [3]:
g = nx.karate_club_graph()
Sigma(g)
# If this does not display anything, try running this command in another cell:
# `!jupyter nbextension enable --py --sys-prefix ipysigma`
# Then restart anaconda & your notebook and try again

Sigma(nx.Graph with 34 nodes and 78 edges)

In [4]:
# If everything else fails:
# nx.write_gexf(g, './graph.gexf')
# And go to: https://medialab.github.io/nansi/

## Learning how to use networkx

In [5]:
example_graph = nx.Graph()

In [6]:
example_graph.add_node('John', gender='man')
example_graph.add_node('Mary', gender='woman')
example_graph.add_node('Susan', gender='woman')
example_graph.add_node('Albert', gender='man')

In [7]:
len(example_graph)

4

In [8]:
example_graph.add_edge('John', 'Mary', weight=10)
example_graph.add_edge('John', 'Susan', weight=45)
example_graph.add_edge('John', 'Albert', weight=1)
example_graph.add_edge('Mary', 'Albert', weight=2)

In [9]:
'John' in example_graph

True

In [10]:
'Jack' in example_graph

False

In [11]:
example_graph.order(), example_graph.size()

(4, 4)

In [12]:
list(example_graph.nodes.data())

[('John', {'gender': 'man'}),
 ('Mary', {'gender': 'woman'}),
 ('Susan', {'gender': 'woman'}),
 ('Albert', {'gender': 'man'})]

In [13]:
Sigma(example_graph, node_size=example_graph.degree, node_color='gender', edge_size='weight')

Sigma(nx.Graph with 4 nodes and 4 edges)

## Downloading our tweets

In [14]:
QUERY = '(blocage OR occupation) sciencespo'

In [15]:
scraper = TwitterAPIScraper()

In [16]:
TWEETS = []

for tweet in scraper.search_tweets(QUERY):
    TWEETS.append(tweet)

In [17]:
len(TWEETS)

1292

## Building a bipartite graph

*With two kind of nodes: Twitter users, on one side, and the hashtags they use, on the other.*

In [18]:
graph = nx.Graph()

for tweet in TWEETS:
    user = '@' + tweet['user_screen_name']
    graph.add_node(user, category='user')
    
    for hashtag in tweet['hashtags']:
        hashtag = '#' + hashtag
        graph.add_node(hashtag, category='hashtag')
        graph.add_edge(user, hashtag)
        
graph.order(), graph.size()

(1346, 979)

In [19]:
Sigma(
    graph,
    only_largest_component=True,
    node_size=graph.degree,
    node_color='louvain',
    node_metrics=['louvain']
)

Sigma(nx.Graph with 1,346 nodes and 979 edges)

## Projecting the graph

In [20]:
!pip install pelote

In [21]:
from pelote import monopartite_projection

In [22]:
# Relevant documentation: https://github.com/medialab/pelote#monopartite_projection
hashtag_graph = monopartite_projection(graph, 'hashtag', node_part_attr='category')

In [23]:
hashtag_graph.order(), hashtag_graph.size()

(369, 1504)

In [24]:
Sigma(
    hashtag_graph,
    only_largest_component=True,
    node_size=graph.degree,
    node_color='louvain',
    node_metrics=['louvain']
)

Sigma(nx.Graph with 369 nodes and 1,504 edges)

In [25]:
user_graph = monopartite_projection(graph, 'user', node_part_attr='category')

In [26]:
Sigma(
    user_graph,
    only_largest_component=True,
    node_size=graph.degree,
    node_color='louvain',
    node_metrics=['louvain'],
    edge_size='weight',
    edge_weight='weight'
)

Sigma(nx.Graph with 977 nodes and 16,492 edges)